In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import networkx as nx
import numpy as np
import pandas as pd
import math
import requests
from bs4 import BeautifulSoup
from collections import defaultdict
import re



#  _Ranking Algorithms Behind Modern Web Search_

---

**Abstract**

The primary goal of this project is to explore how modern web search engines work by investigating the mathematical foundations and practical implementations of key algorithms, including **PageRank**, **HITS**, and **BM25**.
The project begins with an overview of the evolution of **web search**, focusing on the role of **Google Search**, and implementation of algorithms. I explore some historical perspective which sets the stage for a deep theoretical and practical dive into the core mechanisms behind these algorithms.
I explore the mathematical underpinnings of link-based ranking systems, including concepts such as **graph theory**, **Markov chains**, **stochastic matrices**, **eigenvectors**, **stationary distributions**, and the **random surfer model**. Each algorithm is implemented from scratch using **Python**, illustrating their logic, behavior, and impact on page ranking and information retrieval.
To support **understanding**, the project includes:

- Clear mathematical formulations

- Code walkthroughs and Python implementations

- Visualizations and Images

- Comparative analysis of algorithm behavior


This project aims to provide a comprehensive and comparative view of how modern search algorithms function work and why they remain foundational to web search technologies today. It serves a technical deep dive into information retrieval algorithms,testing algorithms and a tribute to the innovations that shaped the way we find information online.



---

# **Introduction**

## **What is Web Search?**

Web search refers to the process of finding information on the World Wide Web through **search engines**. When a user enters a query (typically a word or phrase), a search engine returns a list of relevant web pages ranked by relevance and importance.
Definition of the web search we can say it is branch of **Information Retrieval (IR)** focused on **indexing**, **crawling**, **ranking**, and **retrieving web content** in response to user queries.


### **Search Engine**
A search engine is a software system that provides hyperlinks to web pages and other relevant information on the Web in response to a user's query.
The user inputs a query within a web browser or a mobile app, and the search results are often a list of hyperlinks, accompanied by textual summaries and images. 
Users also have the option of limiting the search to a specific type of results, such as images, videos, or news.

For a search provider, its engine is part of a distributed computing system that can encompass many data centers throughout the world. 
The speed and accuracy of an engine's response to a query is based on a complex system of **indexing** that is continuously updated by automated web crawlers.\


There have been many search engines since the dawn of the Web in the 1990s, but **Google Search** became the dominant one in the 2000s and has remained so.
It currently has a **90% global market share**.


## **How Web Search Works?** 
A search engine maintains the following processes in near real time:

- **Crawling**
- **Indexing**
- **Processing**
- **Ranking**
- **Searching**


<center>
    <img src="https://media.licdn.com/dms/image/D4D12AQG322OLRq4Ssg/article-cover_image-shrink_600_2000/0/1719918145487?e=2147483647&v=beta&t=ik5wi9nkxhus55h7ST1vlBEMQCI05Yjv4HluUHO_UJw" alt="Image of How Search Engines Work" width="300" height="200">
</center>

## Crawling
Web search engines get their information by **web crawling** from site to site. The **spider** checks for the standard filename **robots.txt**, addressed to it. 
The **robots.txt** file contains directives for search spiders, telling it which pages to crawl and which pages not to crawl. 
After checking for **robots.txt** and either finding it or not, the spider sends certain information back to be indexed depending on many factors, such as the titles, page content, JavaScript, Cascading Style Sheets (CSS), headings, or its metadata in HTML meta tags. 
After a certain number of pages crawled, amount of data indexed, or time spent on the website, the spider stops crawling and moves on. 
No web crawler may actually crawl the entire reachable web. 
Due to infinite websites, spider traps, spam, and other exigencies of the real web, crawlers instead apply a crawl policy to determine when the crawling of a site should be deemed sufficient.
Some websites are crawled exhaustively, while others are crawled only partially.

## Indexing
Indexing means associating words and other definable tokens found on web pages to their domain names and HTML-based fields.
The associations are stored in a public database and accessible through web search queries. 
A query from a user can be a single word, multiple words or a sentence. 
The index helps find information relating to the query as quickly as possible.
Some of the techniques for indexing, and caching are trade secrets, whereas web crawling is a straightforward process of visiting all sites on a systematic basis.


## Searching
Typically when a user enters a query into a search engine it is a few keywords.
The index already has the names of the sites containing the keywords, and these are instantly obtained from the index. 
The real processing load is in generating the web pages that are the search results list.Every page in the entire list must be weighted according to information in the indexes.
Then the top search result item requires the lookup, reconstruction, and markup of the snippets showing the context of the keywords matched. 
These are only part of the processing each search results web page requires, and further pages (next to the top) require more of this post-processing.
Beyond simple keyword lookups, search engines offer their own **GUI**- or command-driven operators and search parameters to refine the search results. 
These provide the necessary controls for the user engaged in the feedback loop users create by filtering and weighting while refining the search results,
given the initial pages of the first search results.


Most search engines support the use of the Boolean operators AND, OR and NOT to help end users refine the search query. 
Boolean operators are for literal searches that allow the user to refine and extend the terms of the search. 
The engine looks for the words or phrases exactly as entered. 
Some search engines provide an advanced feature called **proximity search**, which allows users to define the distance between keywords.
There is also **concept-based** searching where the research involves using statistical analysis on pages containing the words or phrases you search for.
The usefulness of a search engine depends on the relevance of the result set it gives back.


While there may be millions of web pages that include a particular word or phrase, some pages may be more relevant, popular, or authoritative than others. 
Most search engines employ methods to rank the results to provide best results first. 
How a search engine decides which pages are the best matches, and what order the results should be shown in, varies widely from one engine to another.
The methods also change over time as Internet usage changes and new techniques evolve. 
There are two main types of search engine that have evolved: one is a **system of predefined and hierarchically ordered keywords** that humans have programmed extensively.
The other is a system that generates an **inverted index** by analyzing texts it locates. 
This first form relies much more heavily on the computer itself to do the bulk of the work.


Most Web search engines are commercial ventures supported by advertising revenue and thus some of them allow advertisers to have their listings ranked higher in search results for a fee. 
Search engines that do not accept money for their search results make money by running search related ads alongside the regular search engine results. 
The search engines make money every time someone clicks on one of these ads.



---

## **Implementing Basic Crawling**

In [11]:
def crawl(url):
    response = requests.get(url)
    hrefs = []
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        for link in soup.find_all('a'):
            href = link.get('href')
            hrefs.append(href)
    else:
        print(f"Error crawling {url}: {e}")   
        
    return hrefs

In [12]:
books_result =  crawl('https://books.toscrape.com/')

In [13]:
quotes_result = crawl('https://quotes.toscrape.com/')

## **Implementing Crawling and Indexing**

In [15]:
# Inverted index: word → list of URLs
inverted_index = defaultdict(set)

"""tokenize function splits text to lower case and return list of matches of the regex"""

def tokenize(text):
    return re.findall(r'\b\w+\b', text.lower())\


"""crawl_and_index function make fetch request and if it succesfull:
1.parse html,
2.extract visible text,
3.tokenize words
4.Adds the current url to the index entry for each word.
"""

def crawl_and_index(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            text = soup.get_text()
            words = tokenize(text)
            for word in words:
                inverted_index[word].add(url)
    except Exception as e:
        print(f"Error crawling {url}: {e}")



# Example: Crawling two book pages
urls = [
    "https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html",
    "https://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html"
]

for url in urls:
    crawl_and_index(url)

# Example: Find all pages containing the word "book"
print("Pages with 'book':", inverted_index["book"])


Pages with 'book': {'https://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html'}


## **Searching Function**

In [17]:
def search(query):
    query_words = tokenize(query)
    if not query_words:
        return set()

    # Start with the set of URLs for the first word
    results = inverted_index.get(query_words[0], set()).copy()

    # Intersect with the URL sets for the rest of the words (AND search)
    for word in query_words[1:]:
        results &= inverted_index.get(word, set())

    return results

Combine all together and make **simple search engine**!

In [ ]:
import requests
from bs4 import BeautifulSoup
from collections import defaultdict
import re

# Inverted index: maps each word → set of URLs that contain it
inverted_index = defaultdict(set)

# tokenize function: splits text into lowercase words using regex
def tokenize(text):
    return re.findall(r'\b\w+\b', text.lower())

# crawl_and_index function:
# 1. fetches HTML from a URL
# 2. extracts visible text
# 3. tokenizes the text into words
# 4. adds the URL to the index for each word
def crawl_and_index(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            text = soup.get_text()
            words = tokenize(text)
            for word in words:
                inverted_index[word].add(url)
    except Exception as e:
        print(f"Error crawling {url}: {e}")

# Search function: returns URLs containing ALL query words
def search(query):
    query_words = tokenize(query)
    if not query_words:
        return set()

    # Start with the set of URLs for the first word
    results = inverted_index.get(query_words[0], set()).copy()

    # Intersect with the URL sets for the rest of the words (AND search)
    for word in query_words[1:]:
        results &= inverted_index.get(word, set())

    return results

# Example: Crawl some book pages
urls = [
    "https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html",
    "https://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html"
]

for url in urls:
    crawl_and_index(url)

# Command-line search loop
while True:
    query = input("\nEnter search query (or 'exit'): ").strip()
    if query.lower() == 'exit':
        break
    result_urls = search(query)
    if result_urls:
        print("Search results:")
        for url in result_urls:
            print("-", url)
    else:
        print("No results found.")


## **History**
### **Before 1990**
In **1945**, **Vannevar Bush** described an information retrieval system that would allow a user to access a great expanse of information, all at a single desk.
He called it a **memex**.The memex was intended to give a user the capability to overcome the ever-increasing difficulty of locating information in ever-growing centralized indices of scientific work. 
Vannevar Bush envisioned libraries of research with connected annotations, which are similar to modern hyperlinks.
Link analysis nowadays are crucial component of search engines through algorithms such as **PageRank**.

### **1990 Birth of search engines**
The first well documented search engine that searched content files  was **Archie**, which debuted on **10 September 1990.**

<center>
    <img src="https://hackaday.com/wp-content/uploads/2024/05/Archie_search_engine_thumb.jpg?w=600&h=600" alt="Image of Archie search engine" width="300" height="200">
</center>


It was created by Alan Emtage, computer science student at McGill University in Montreal, Quebec, Canada. 
The program downloaded the directory listings of all the files located on public anonymous FTP (File Transfer Protocol) sites, creating a searchable database of file names; 
However, Archie Search Engine did not index the contents of these sites since the amount of data was so limited it could be readily searched manually.


After **Archie** was **Veronica** and **Jughead**!

#### Veronica and Jughead

Like **Archie**, they searched the file names and titles stored in **Gopher**(is s a communication protocol designed for distributing, searching, and retrieving documents in Internet Protocol networks) index systems.
**Veronica** provided a keyword search of most Gopher menu titles in the entire Gopher listings. 
**Jughead**  was a tool for obtaining menu information from specific Gopher servers. 

#### Yahoo

The first popular search engine on the Web was **Yahoo!** The first product from Yahoo!, founded by Jerry Yang and David Filo in January 1994, 
was a Web directory called **Yahoo! Directory**. In 1995, a search function was added, allowing users to search Yahoo! Directory.
It became one of the most popular ways for people to find web pages of interest, but its search function operated on its web directory, 
rather than its full-text copies of web pages.

After some years of creation search engines ,**PageRank** was born!

### **History of PageRank**
A search engine called **RankDex** from IDD Information Services, designed by **Robin Li** in 1996, developed a strategy for site-scoring and page-ranking.
It was one of the first search engines to use link analysis-which is ranking the popularity of a web site based on how many other sites had linked to it for ranking pages meaning it also considered hyperlink structure, not just keywords.
It is called **RankDex** and it was launched in 1996.**Li** filed a patent for the technology in **RankDex** in 1997.
He later used it when he founded **Baidu** in China in 2000.
Google founder **Larry Page** referenced Li's work as a citation in some of his U.S. patents for **PageRank**.


**Larry Page** and **Sergey Brin** developed **PageRank** at Stanford University in 1996 as part of a research project about a new kind of search engine. Sergey Brin had the idea that information on the web could be ordered in a hierarchy by link popularity-a page ranks higher as there are more links to it.The system was developed with the help of Scott Hassan and Alan Steremberg, both of whom were cited by Page and Brin as being critical to the development of Google. Rajeev Motwani and Terry Winograd co-authored with Page and Brin the first paper about the project, describing **PageRank** and the initial prototype of the [Google search engine](https://en.wikipedia.org/wiki/Google_Search), published in 1998.
> 📜 [*The Anatomy of a Large-Scale Hypertextual Web Search Engine*](http://infolab.stanford.edu/pub/papers/google.pdf)


While just nowadays has many factors that determine the ranking of Google search results, **PageRank** continues to provide the basis for all of Google's web-search tools.The name **PageRank** plays on the name of developer **Larry Page**, as well as of the concept of a web page.The word is a trademark of Google, and the **PageRank** process has been patented assigned to Stanford University and not to Google.Google has exclusive license rights on the patent from Stanford University. The university received 1.8 million shares of Google in exchange for use of the patent.It sold the shares in 2005 for $336 million.

## **The Problem of Web Search Before PageRank?**

Before Google and the introduction of PageRank, web search engines struggled to deliver high-quality, relevant results. 
As the internet rapidly expanded during the 1990s, the number of web pages grew into the millions, then billions — but search technology failed to keep up with that growth in a meaningful way.
Most early search engines  relied heavily on simple keyword matching to retrieve results. 
They would look for pages that contained the same words as the user's query, and rank them based on factors like:
frequency of the keyword,location of the keyword,basic metadata.While this approach was easy to implement, it had major weaknesses:it was easy to manipulate ,it treated all pages as equally trustworthy or important and it didn’t consider how humans value content — through references and links.
There was no good way to determine which pages were actually important or trustworthy. 


A personal blog and a university website could rank equally if they both mentioned the same keywords.
This meant that relevance and quality often suffered.Search engines needed a way to filter spammy or low-quality pages,highlight content that was trusted or referenced by others,deliver more objective and useful results for users.
This set the stage for a fundamentally new idea: using link structure to infer page importance — the idea behind **PageRank**.

## 📜 Timeline: Evolution of Web Search and the Birth of PageRank

| Year | Milestone |
|------|-----------|
| **1945** | **memex** |
| **1990** | **Archie** | 
| **1990-1993** | **Veronica and Jughead** | 
| **1995** | **Yahoo** | 
| **1996** | **RankDex (Robin Li)** | 
| **1998** | **PageRank (Page & Brin)** |
| **1998** | **Google Founded** | 

---

Nowadays active search engine crawlers include those of **Google**, **Sogou**, **Baidu**, **Bing**, **Gigablast**, **Mojeek**, **DuckDuckGo** and **Yandex**.

So we already see **What is web search?**.Check **history** and **reason** to have search engines. But one question jump to us!

## **Why Web Search is a Complex Problem?**
While modern web search engines like **Google** seem simple to the user—just type and get answers—they are, in reality, built on one of the most complex engineering and mathematical systems ever developed. Several key challenges make web search incredibly difficult!The internet contains billions of pages and continues to grow every day. 
Search engines must:crawl and store huge amounts of content,
keep the index updated in real-time,
deliver results in milliseconds.Also users usually enter 2–3 word queries. These are often:
ambiguous,
misspelled or vague,
missing key context.Understanding intent from such input requires sophisticated models, often using machine learning and natural language processing.At these days every search has relevence.
What is relevant to one person might not be to another.
Also Web contains both valuable information and misleading content. Search engines must:
detect spam and low-quality pages,
rank trustworthy, authoritative content higher,
prevent manipulation. And many other complex things!
The problem of web search arose from the need to navigate and extract meaning from the overwhelming volume of online content. 
The complexity of this task grows with the internet itself and demands continuous innovation in areas like graph theory, information retrieval, AI, and large-scale systems. 
Solving this problem has not only changed how we access information—but transformed society itself. 
Understanding the algorithms behind it gives us insight into one of the most powerful and influential technologies of the modern age.

---

## **The Anatomy of a Large-Scale Hypertextual Web Search Engine (1998)**

This is all important info from _The Anatomy of a Large-Scale Hypertextual Web Search Engine (1998)_ and show how **Google** works!

As I mentioned a little while ago in 1998, Larry Page and Sergey Brin published a paper titled **The Anatomy of a Large-Scale Hypertextual Web Search Engine.** This work introduced **Google** to the world and laid the foundation for modern web search engines.
At the heart of their prototype was a novel ranking algorithm called **PageRank**, which changed the way search results were ordered on the web.
They present **Google**, a prototype of a large-scale search engine which makes heavyuse of the structure present in hypertext designed to **crawl** and **index* the Web efficiently and produce much more satisfying search results than existing systems.


Search engines index tens to hundreds of millions of web pages involving a comparable number of distinct terms.
They answer tens of millions of queries every day. 
Despite the importance of large-scale search engines on the web,very little academic research has been done on them.
Furthermore, due to rapid advance in technology and web proliferation, creating a web search engine today is very different from three years ago.
Apart from the problems of scaling traditional search techniques to data of this magnitude, there are new technical challenges involved with using the additional information present in hypertext to produce better search results.
They addresses this question of how to build a practical large-scale system which can exploit the additional information present in hypertext. 
Also look at the problem of how to effectively deal with uncontrolled hypertext collections where anyone can publish anything they want.


The web creates new challenges for information retrieval. The amount of information on the web is
growing rapidly, as well as the number of new users inexperienced in the art of web research. People are
likely to surf the web using its **link graph**, often starting with high quality human maintained indices. 
Human maintained lists cover popular topics effectively but are
subjective, expensive to build and maintain, slow to improve, and cannot cover all esoteric topics.


Automated search engines that rely on keyword matching usually return too many low quality matches.
To make matters worse, some advertisers attempt to gain people’s attention by taking measures meant to
mislead automated search engines. They have built a large-scale search engine which addresses many of
the problems of existing systems. It makes especially heavy use of the additional structure present in
hypertext to provide much higher quality search results. 


They said they chose their system name, **Google**, because it
is a common spelling of googol, or $10^{100}$ and fits well with they goal of building very large-scale search
engines. 

Search engine technology has had to scale dramatically to keep up with the growth of the web. In 1994,
one of the **first web search engines**, the **World Wide Web Worm (WWWW)** had an index
of **110,000** web pages and web accessible documents. As of November, 1997, the top search engines
claim to index from **2 million (WebCrawler) to 100 million web documents (from Search Engine
Watch)**.The number of queries search engines handle has grown incredibly
too. In March and April 1994, the World Wide Web Worm received an average of about **1500 queries
per day**. In November 1997, reach **20 million** queries per day. 


With the increasing number of users on the web, and automated systems which query search engines, it is likely
that top search engines will handle hundreds of millions of queries per day. The goal of
they system is to address many of the problems, both in **quality** and **scalability**, introduced by scaling
search engine technology to such extraordinary numbers.


Creating a search engine which scales even to today’s web presents many challenges. 
- **Fast crawling**technology is needed to gather the web documents and keep them up to date.
- **Storage space** must be used efficiently to store indices and, optionally, the documents themselves.
- The **indexing system** must process hundreds of gigabytes of data efficiently.
- **Queries** must be handled quickly, at a rate of hundreds to thousands per second.


These tasks are becoming increasingly difficult as the Web grows. In designing Google,
they have considered both the rate of growth of the Web and technological changes. They said Google is designed to
scale well to extremely **large data sets**. It makes efficient use of storage space to **store the index**. Its data
structures are optimized for fast and efficient access.


They main goal is to improve the **quality** of web search engines. Some people believed that a
complete search index would make it possible to find anything easily. According to Best of the Web
1994 they said: The best navigation service should make it easy to find almost anything on the
web once all the data is entered!


Anyone who has used a search engine recently, can readily testify that the completeness of the index is not the only factor in
the quality of search results. **Junk results** often wash out any results that a user is interested in.


In fact,November 1997, one of the top four commercial search engines finds itself returns its own
search page in response to its name in the top ten results. One of the main causes of this problem is that
the number of documents in the indices has been increasing by many orders of magnitude, but the user’s
ability to look at documents has not. People are still only willing to look at the first few tens of results.
Because of this, as the collection size grows, they need tools that have very **high precision**.Indeed, they want their notion of **relevant** to
only include the very best documents since there may be tens of thousands of slightly relevant
documents. This very high precision is important even at the expense of recall (the total number of
relevant documents the system is able to return). 


There is quite a bit of recent optimism that the use of
more hypertextual information can help improve search and other applications.In particular, link structure  and link text provide a lot of
information for making relevance judgments and quality filtering. Google makes use of both link
structure and anchor text.


Aside from tremendous growth, the Web has also become increasingly commercial over time. In 1993,
1.5% of web servers were on .com domains. This number grew to over 60% in 1997. At the same time,
search engines have migrated from the academic domain to the commercial. Up until now most search
engine development has gone on at companies with little publication of technical details. This causes
search engine technology to remain largely a black art and to be advertising oriented.
They important design goal was to build systems that reasonable numbers of people can actually use.
Usage was important to them because they think some of the most interesting research will involve
leveraging the vast amount of usage data that is available from modern web systems.It is very difficult to get data,
mainly because it is considered commercially valuable.
Final design goal was to build an architecture that can support novel research activities on
large-scale web data. To support novel research uses, Google stores all of the actual documents it crawls
in **compressed form**. They designing Google to set up an environment where
other researchers can come in quickly, process large chunks of the web, and produce interesting results
that would have been very difficult to produce otherwise. In the short time the system has been up, there
have already been several papers using databases generated by Google, and many others are underway.


The Google search engine has two important features that help it produce high precision results. First, it
makes use of the **link structure** of the Web to calculate a quality ranking for each web page. This ranking
is called **PageRank**. Second, Google utilizes link to improve
search results. 

They created maps containing **518 million** of these hyperlinks, a
**significant sample** of the total. These maps allow rapid calculation of a web page’s **PageRank**(an
objective measure of its citation importance that corresponds well with people’s subjective idea of
importance). Because of this correspondence, **PageRank** is an excellent way to **prioritize the results** of
web keyword searches. For most popular subjects, a simple text matching search that is restricted to web
page titles performs admirably when PageRank prioritizes the results.

#### **PageRank**

Academic citation literature has been applied to the web, largely by counting citations or backlinks to a
given page. This gives some approximation of a page’s importance or quality. PageRank extends this
idea by not counting links from all pages equally, and by normalizing by the number of links on a page.

*We assume page A has pages T1...Tn which point to it. The parameter d
is a damping factor which can be set between 0 and 1. We usually set d to **0.85**. Also C(A) is defined as the number of links going
out of page A. The PageRank of a page A is given as follows:*

$$PR(A) = (1-d) + d (PR(T1)/C(T1) + ... + PR(Tn)/C(Tn))$$

**PageRanks** form a **probability distribution** over web pages, so the sum of all web pages’ PageRanks will be one.
**PageRank** or PR(A) can be calculated using a simple iterative algorithm, and corresponds to the
principal **eigenvector** of the normalized **link matrix** of the web. 


PageRank can be thought of as a model of user behavior. We assume there is a **random surfer** who is
given a web page at random and keeps clicking on links, **never hitting back** but eventually gets bored
and starts on another random page. The probability that the random surfer visits a page is its **PageRank**.


**Damping factor** is the probability at each page the **random surfer** will get bored and request
another random page. 


Intuitive justification is that a page can have a high **PageRank** if there are many pages that point
to it, or if there are some pages that point to it and have a high **PageRank**. Intuitively, pages that are well
cited from many places around the web are worth looking at.


#### **Link Texts**
The text of links is treated in a special way in search engine. Most search engines associate the text
of a link with the page that the link is on. They associate it with the page the link points to.
This has several advantages. First, anchors often provide more accurate descriptions of web pages than
the pages themselves. Second, anchors may exist for documents which cannot be indexed by a
text-based search engine, such as images, programs, and databases. This makes it possible to return web
pages which have not actually been crawled. Note that pages that have not been crawled can cause
problems, since they are never checked for validity before being returned to the user. In this case, the
search engine can even return a page that never actually existed, but had hyperlinks pointing to it.
However, it is possible to sort the results, so that this particular problem rarely happens.
This idea of propagating anchor text to the page it refers to was implemented in the World Wide Web
Worm especially because it helps search non-text information, and expands the search
coverage with fewer downloaded documents. We use anchor propagation mostly because anchor text
can help provide better quality results. Using anchor text efficiently is technically difficult because of
the large amounts of data which must be processed. In our current crawl of 24 million pages, we had
over 259 million anchors which we indexed. 


### **Google Architecture Overview**
In this section, they give a high level overview of how
the whole system works.


<center>
    <img src="https://www.researchgate.net/profile/Nureni-Azeez/publication/255644944/figure/download/fig1/AS:285567700615168@1445096053801/High-Level-Google-Architecture.png" alt="Image of High Level Google Arhitecture" width="300" height="200">
</center>


Most of **Google** is
implemented in C or C++ for efficiency and can run in
either Solaris or Linux.
In Google, the **web crawling** (downloading of web pages)
is done by several distributed crawlers. There is a
**URLserver** that sends lists of **URLs** to be fetched to the
crawlers. The web pages that are fetched are then sent to
the **storeserver**. The storeserver then **compresses** and **stores**
the web pages into a **repository**. Every web page has an
associated ID number called a **docID** which is assigned
whenever a new URL is parsed out of a web page. The
**indexing function** is performed by the **indexer** and the
**sorter**. The **indexer** performs a number of functions. It reads
the repository, uncompresses the documents, and parses them. Each document is converted into a set of
word occurrences called **hits**. The **hits** record the word, position in document, an approximation of font
size, and capitalization. The **indexer** distributes these hits into a set of **barrels**, creating a partially
sorted forward index. The indexer performs another important function. It parses out all the links in
every web page and stores important information about them in an **anchors file**. This file contains
enough information to determine where each link points from and to, and the text of the link.
The **URLresolver** reads the anchors file and converts relative URLs into absolute URLs and in turn into
**docIDs**. It puts the anchor text into the forward index, associated with the **docID** that the anchor points
to. It also generates a database of links which are pairs of **docIDs**. The **links database** is used to compute
**PageRanks** for all the documents.


The **sorter** takes the **barrels**, which are sorted by **docID** and
resorts them by **wordID** to generate the **inverted index**. This is done in place so that little temporary
space is needed for this operation. The sorter also produces a list of **wordIDs** and **offsets** into the
**inverted index**. A program called **DumpLexicon** takes this list together with the lexicon produced by the
indexer and generates a new lexicon to be used by the searcher. The **searcher** is run by a web server and
uses the lexicon built by **DumpLexicon** together with the **inverted index** and the **PageRanks** to answer
queries. 


**Google’s data structures** are optimized so that a large document collection can be **crawled**, **indexed**, and
**searched** with little cost.


#### **Repositary**

The **repository** contains the full HTML of every web page.
Each page is compressed using **zlib**. The
choice of compression technique is a tradeoff between speed
and compression ratio. They chose zlib’s speed over a
significant improvement in compression offered by bzip.In the
repository, the documents are stored one after the other and
are prefixed by **docID**, **length**, and **URL**.
The repository requires no other data structures to be used in order to access it. This helps with
data consistency and makes development much easier;

**we can rebuild all the other data structures from
only the repository and a file which lists crawler errors**. 

#### **Document Index**
The document index keeps information about each document. It is a fixed width **ISAM** (Index sequential
access mode) index, ordered by **docID**. The information stored in each entry includes the current
document status, a pointer into the repository, a document checksum, and various statistics. If the
document has been crawled, it also contains a pointer into a variable width file called **docinfo** which
contains its URL and title. Otherwise the pointer points into the **URLlist** which contains just the URL.
This design decision was driven by the desire to have a reasonably compact data structure, and the
ability to fetch a record in one disk seek during a search.
Additionally, there is a file which is used to convert **URLs** into **docID**s. It is a list of URL checksums
with their corresponding docIDs and is sorted by checksum. In order to find the docID of a particular
URL, the URL’s checksum is computed and a binary search is performed on the checksums file to find
its docID. URLs may be converted into docIDs in batch by doing a merge with this file. This is the
technique the URLresolver uses to turn URLs into docIDs. This batch mode of update is crucial because
otherwise we must perform one seek for every link which assuming one disk would take more than a
month for our **322 million** link dataset. 


#### **Hit List**
A hit list corresponds to a list of occurrences of a particular word in a particular document including
position, font, and capitalization information. Hit lists account for most of the space used in both the
forward and the inverted indices. Because of this, it is important to represent them as efficiently as
possible. Тhey considered several alternatives for encoding position, font, and capitalization -- simple
encoding (a triple of integers), a compact encoding (a hand optimized allocation of bits), and **Huffman
coding**. In the end we chose a **hand optimized compact encoding ** since it required far less space than the
simple encoding and far less bit manipulation than **Huffman coding**.


They compact encoding uses two bytes for every hit. There are two types of hits: **fancy hits** and **plain hits**.
**Fancy hits** include hits occurring in a **URL**, **title**, **anchor text**, or **meta tag**. **Plain hits** include everything
else. A plain hit consists of a **capitalization bit**, **font size**, and **12 bits** of word position in a document. 
**Font size** is represented relative to the rest of the document
using **three bits** . A fancy hit consists of a capitalization bit, the **font size set to 7** to indicate it is a fancy hit, **4 bits to encode the
type of fancy hit**, and **8 bits of position**. For **anchor hits**, the 8 bits of position are split into 4 bits for
position in anchor and 4 bits for a hash of the docID the anchor occurs in. This gives us some limited
phrase searching as long as there are not that many anchors for a particular word. They expect to update
the way that anchor hits are stored to allow for greater resolution in the position and docIDhash fields.


They use font size relative to the rest of the document because when searching, you do not want to rank
otherwise identical documents differently just because one of the documents is in a larger font.

The length of a hit list is stored before the hits themselves.
To save space, the length of the hit list is combined with the
wordID in the forward index and the docID in the inverted
index. This limits it to 8 and 5 bits respectively . If the length is longer than would fit in that many
bits, an escape code is used in those bits, and the next two
bytes contain the actual length. 




#### **Crawling the Web**
Running a web crawler is a challenging task. There are tricky performance and reliability issues and
even more importantly, there are social issues. Crawling is the most fragile application since it involves
interacting with hundreds of thousands of web servers and various name servers which are all beyond
the control of the system.
In order to scale to hundreds of millions of web pages, Google has a fast distributed crawling system. A
single **URLserver** serves **lists of URLs** to a number of crawlers.Each crawler keeps roughly 300 connections
open at once. This is necessary to retrieve web pages at a fast enough pace. At peak speeds, the system
can crawl over 100 web pages per second using four crawlers. This amounts to roughly 600K per second
of data. A major performance stress is **DNS** lookup. Each crawler maintains a its own **DNS cache** so it
does not need to do a DNS lookup before crawling each document. Each of the hundreds of connections
can be in a number of different states: **looking up DNS**, **connecting to host**, **sending request**, and
**receiving response**. These factors make the crawler a complex component of the system. It uses
asynchronous IO to manage events, and a number of queues to move page fetches from state to state.
It turns out that running a crawler which connects to more than half a million servers, and generates tens
of millions of log entries generates a fair amount of email and phone calls. Because of the vast number
of people coming on line, there are always those who do not know what a crawler is, because this is the
first one they have seen. Almost daily, we receive an email something like, "Wow, you looked at a lot of
pages from my web site. How did you like it?" There are also some people who do not know about the
robots exclusion protocol, and think their page should be protected from indexing by a statement like,
"This page is copyrighted and should not be indexed", which needless to say is difficult for web crawlers
to understand. 


Also, because of the huge amount of data involved, unexpected things will happen. Because of the immense variation in web pages and
servers, it is virtually impossible to test a crawler without running it on large part of the Internet.
Invariably, there are hundreds of obscure problems which may only occur on one page out of the whole
web and cause the crawler to crash, or worse, cause unpredictable or incorrect behavior. Systems which
access large parts of the Internet need to be designed to be very robust and carefully tested. Since large
complex systems such as crawlers will invariably cause problems, there needs to be significant resources
devoted to reading the email and solving these problems as they come up.

#### **Indexing The Web**

- **Parsing --** Any parser which is designed to run on the entire Web must handle a huge array of
possible errors. These range from typos in HTML tags to kilobytes of zeros in the middle of a tag,
non-ASCII characters, HTML tags nested hundreds deep, and a great variety of other errors that
challenge anyone’s imagination to come up with equally creative ones. For maximum speed they use flex to generate a lexical analyzer which
we outfit with its own stack. Developing this parser which runs at a reasonable speed and is very
robust involved a fair amount of work. 
- **Indexing--** After each document is parsed, it is encoded into a number
of barrels. Every word is converted into a wordID by using an in-memory hash table -- the lexicon.
New additions to the lexicon hash table are logged to a file. Once the words are converted into
wordID’s, their occurrences in the current document are translated into hit lists and are written into
the forward barrels. The main difficulty with parallelization of the indexing phase is that the
lexicon needs to be shared. Instead of sharing the lexicon, they took the approach of writing a log of
all the extra words that were not in a base lexicon, which they fixed at 14 million words. That way
multiple indexers can run in parallel and then the small log file of extra words can be processed by
one final indexer.
- **Sorting --** In order to generate the inverted index, the sorter takes each of the forward barrels and
sorts it by wordID to produce an inverted barrel for title and anchor hits and a full text inverted
barrel. This process happens one barrel at a time, thus requiring little temporary storage. They
parallelize the sorting phase to use as many machines as they have simply by running multiple
sorters, which can process different buckets at the same time. Since the barrels don’t fit into main
memory, the sorter further subdivides them into baskets which do fit into memory based on
**wordID** and **docID**. Then the sorter, loads each basket into memory, sorts it and writes its contents
into the short inverted barrel and the full inverted barrel.

#### **Searching**
The goal of searching is to provide quality search results efficiently. Many of the large commercial
search engines seemed to have made great progress in terms of efficiency. Therefore, they have focused
more on quality of search in their research, although they believe their solutions are scalable to commercial
volumes with a bit more effort.

*Google Query Evaluetion*

- 1. Parse the query.

- 2. Convert words into wordIDs.

- 3. Seek to the start of the doclist in
the short barrel for every word.

- 4. Scan through the doclists until
there is a document that matches
all the search terms.

- 5. Compute the rank of that
document for the query.

- 6. If we are in the short barrels and at
the end of any doclist, seek to the
start of the doclist in the full barrel
for every word and go to step 4.

- 7. If we are not at the end of any
doclist go to step 4.
Sort the documents that have
matched by rank and return the top k.


#### **Ranking System**
Google maintains much more information about web
documents than typical search engines. Every hitlist
includes position, font, and capitalization information.
Additionally, they factor in hits from anchor text and the
PageRank of the document. Combining all of this
information into a rank is difficult. They designed their ranking
function so that no particular factor can have too much
influence.


Google considers each hit to be one of several different types (**title**, **anchor**, **URL**, **plain text**, **large font**,
**plain text small font**, **...**), each of which has its own type-weight. The type-weights make up a vector
indexed by type. Google counts the number of hits of each type in the hit list. Then every count is
converted into a count-weight. Count-weights increase linearly with counts at first but quickly taper off
so that more than a certain count will not help.


They take the dot product of the vector of count-weights
with the vector of type-weights to compute an IR score for the document. Finally, the **IR score** is
combined with **PageRank** to give a **final rank** to the document.


For a multi-word search, the situation is more complicated. Now multiple hit lists must be scanned
through at once so that hits occurring close together in a document are weighted higher than hits
occurring far apart. The hits from the multiple hit lists are matched up so that nearby hits are matched
together. For every matched set of hits, a proximity is computed. The proximity is based on how far
apart the hits are in the document (or anchor) but is classified into 10 different value **bins** ranging from
a phrase match to **not even close**. Counts are computed not only for every type of hit but for every type
and proximity. Every type and proximity pair has a type-prox-weight. The counts are converted into
count-weights and we take the dot product of the count-weights and the type-prox-weights to compute
an IR score. All of these numbers and matrices can all be displayed with the search results using a
special debug mode. These displays have been very helpful in developing the ranking system. 

#### **Feedback**
The ranking function has many parameters like the type-weights and the type-prox-weights. Figuring out
the right values for these parameters is something of a black art. In order to do this, we have a user
feedback mechanism in the search engine. A trusted user may optionally evaluate all of the results that
are returned. This feedback is saved. Then when we modify the ranking function, we can see the impact
of this change on all previous searches which were ranked. Although far from perfect, this gives us some idea of how a change in the ranking function affects the search results.

#### **Results and Performance**
The most important measure of a search
engine is the quality of its search results.
While a complete user evaluation is
beyond the scope of this paper, their own
experience with Google has shown it to
produce better results than the major
commercial search engines for most
searches. 
All results are reasonably high
quality pages and, at last check, none
were broken links. This is largely because
they all have high **PageRank**. Of
course a true test of the quality of a search engine would involve an extensive user study or results
analysis.

#### **Search Performance**

Current version of Google answers most queries in between 1 and 10 seconds. Furthermore,
Google does not have any optimizations such as query caching, subindices on common terms, and other
common optimizations. They intend to speed up Google considerably through distribution and hardware,
software, and algorithmic improvements.

Google is designed to be a scalable search engine.
The primary goal is to provide high quality search
results over a rapidly growing World Wide Web.
Google employs a number of techniques to improve
search quality including **page rank**, **anchor text**, and
**proximity information**.Google is a
complete architecture for gathering web pages,
indexing them, and performing search queries over
them.


---

## **Needed Math Theory**


What is fascinating with the PageRank algorithm is how to start from a complex problem and end up with a very simple solution.  I will give the idea and theory behind the **PageRank** algorithm.

### **Random Walk**
The web can be represented like a directed graph where nodes represent the web pages and edges form links between them. Typically, if a node (web page) **i** is linked to a node **j**, it means that **i** refers to **j**.

<center>
    <img src="https://iq.opengenus.org/content/images/2020/05/IMG_0023.jpg" alt="Directed Graph" width="300" height="200">
</center>


We have to define what is the importance of a web page. As a first approach, we could say that it is the total number of web pages that refer to it. If we stop to this criteria, the importance of these web pages that refer to it is not taken into account. In other words, an important web page and a less important one has the same weight. Another approach is to assume that a web page spread its importance equally to all web pages it links to.By doing that, we can then define the score  of a node **j** as follows:

$$r_j = \sum_{i \mapsto j}\frac{r_i}{di} $$

where $r_i$ is the score of the node **i** and $d_i$ its out-degree.


From the example above, we can write this linear system:

$$
\begin{cases}
r_0 = \frac{r_4}{3} \\
r_1 = \frac{r_2}{2} + \frac{r_4}{3} + r_3 \\
r_2 = \frac{r_0}{3} + \frac{r_4}{3} \\
r_3 = \frac{r_2}{2} + \frac{r_0}{3} \\
r_4 = \frac{r_0}{3} + r_1 \\
\end{cases}
$$


By passing the right-hand side of this linear system into the left-hand side, we get a new linear system that we can solve by using Gaussian elimination. But this solution is limited for small graphs. Indeed, as this kind of graphs are sparse and Gauss elimination modifies the matrix when performing its operations, we lose the sparsity of the matrix and it would take more memory space. In the worst case, the matrix can no longer be stored.


---

### **Damping Factor**
The **PageRank theory** holds that an imaginary surfer who is randomly clicking on links will eventually stop clicking. The probability, at any step, that the person will continue following links is a damping factor **d**. The probability that they instead jump to any random page is **1 - d**. Various studies have tested different damping factors, but it is generally assumed that the damping factor will be set around **0.85**.

---

### **What Is an Adjacency Matrix?**
An **adjacency matrix** is a mathematical representation of a graph. It tells you which nodes (web pages, in the case of PageRank) are connected to each others.It’s purely structural — just records the presence of links.
For a graph with **𝑛** nodes, the adjacency matrix **𝐴** is an **𝑛×𝑛**matrix where:

$ A_{ij} = 1$ if there **is a link** from node **i** to node **j**


$ A_{ij} = 0$ if there **is no link** from node **i** to node **j**

Let's give some simple example for good understanding ->

Say we have a graph with 3 nodes:

- **Page A** links to **B**

- **Page B** links to **C**

- **Page C** links to **A**

Then the **adjacency matrix 𝐴**  would be:


$$ A = 
\begin{bmatrix}
0 & 1 & 0 \\
0 & 0 & 1 \\
1 & 0 & 0
\end{bmatrix}$$

- Row 1: **Page A** → **Page B**

- Row 2: **Page B** → **Page C**

- Row 3: **Page C** → **Page A**

### **How Adjacency Matrix change to Transition Probability Matrix?**

We can’t use the raw **adjacency matrix** for PageRank directly. We must transform it into a **stochastic matrix** a matrix where:

- Each column sums to 1

- It represents probabilities of moving from one page to another

$$ M_{ij} =
\begin{cases}
\frac{1}{outer\space links}, & \text{if page j links to page i }\\
0, & \text{otherwise}
\end{cases} $$



$$ M =
\begin{bmatrix}
M_{11} & M_{12} & \cdots & M_{1n} \\
M_{21} & M_{22} & \cdots & M_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
M_{n1} & M_{n2} & \cdots & M_{nn}
\end{bmatrix}$$ 

---

### **What is eigenvector?**

An eigenvector is a special kind of vector in linear algebra. When you multiply a matrix by an eigenvector, the result is just a scaled version of the same vector—not a different direction. This scaling factor is called an **eigenvalue**.

$$ A \cdot \vec{v} = \lambda \cdot \vec{v} $$

- **A** is a square matrix
- **v** is an eigenvector
- **λ** is the eigenvalue

### **How these help in our situation?**
In the PageRank algorithm, you represent the web as a **transition matrix**-**M**, 
where how we say each entry represents the probability of going from one page to another.

Then you try to find a **vector of PageRanks** that satisfies this equation:

$$ M \cdot \vec{r} = \vec{r} $$


$$ M \cdot \vec{r} = 1 \cdot \vec{r} $$

---

This means:
**The PageRank** vector is an eigenvector of the matrix **𝑀**.
**The corresponding eigenvalue is 1**
$ \vec{r}$ represents the steady-state distribution of the random surfer.

After many steps, the probability of the surfer being on each page stabilizes

That stabilized vector is the **eigenvector with eigenvalue 1** — and those are your **PageRank scores**

## **Markov Chains**

A **Markov Chain** is a mathematical model describing a system that moves between different **states** with certain **probabilities**. The key characteristic is the **Markov property**:  
**The next state depends only on the current state, not on the sequence of previous states.**

In the context of web browsing, imagine a user clicking links from one page to another. They don’t remember how they got to the current page—they only consider their **current location** when deciding where to go next.

---


- **States**: Represent individual web pages.
- **Transitions**: Represent links between pages.

- **Transition Probability**: The likelihood of moving from one page to another. This includes the **damping factor** to handle random jumps.
- **Transition Matrix** \( P \): A matrix where each column represents the probabilities of moving from a given page to others.

Example of a simple transition matrix:

$$
M = 
\begin{bmatrix}
0 & 0.5 & 0 \\
1 & 0 & 1 \\
0 & 0.5 & 0
\end{bmatrix}
$$

Each column sums to 1, which is a requirement for **stochastic matrices** used in Markov Chains.

---

Transition Matrix in Practice:

$$
P = \left(\begin{array}{cc}
0 & 0 & 0 & 0 & \frac{1}{3} \\
0 & 0 & \frac{1}{2} & 1 & \frac{1}{3} \\
\frac{1}{3} & 0 & 0 & 0 &  \frac{1}{3} \\
\frac{1}{3} & 0 & \frac{1}{2} & 0 & 0 \\
\frac{1}{3} & 1 & 0 & 0 & 0
\end{array} \right)
$$

This matrix is **column-stochastic**, and its **transpose** is **row-stochastic**, satisfying the conditions to apply **Markov chain theorems**.

---


Let the initial probability distribution be:

$$
\pi^{(0)} = \left( \frac{1}{n}, \frac{1}{n}, \ldots, \frac{1}{n} \right)
$$

At each step:

$$
\pi^{(t+1)} = P \cdot \pi^{(t)}
$$

The distribution evolves until it **converges** to a **stationary distribution** $\pi$, where:

$$
P \cdot \pi = \pi
$$

This represents the long-term probability of being on each page. If the graph is **strongly connected** (every node is reachable from every other), a stationary distribution **always exists**.

---



Solving $ P \cdot \pi = \pi$ is equivalent to finding the **eigenvector** of matrix $ P $ with eigenvalue 1. According to the **Frobenius-Perron Theorem**:

> If $ P $ is a **positive** and **square** matrix, then it has a unique dominant eigenvector with all positive entries.

In our case:
-  $P$ is a positive matrix (after applying damping).
- The dominant eigenvector $\pi$ corresponds to the **PageRank scores**.
- The entries of $\pi$ indicate the **relative importance** of each page.

---

We use the **power iteration method**:

1. Start with an initial guess $ \pi^{(0)} $
2. Update using $ \pi^{(t+1)} = P \cdot \pi^{(t)} $
3. Repeat until convergence

After convergence, the vector $ \pi $ gives the **ranking** of web pages.

The higher the value of $ \pi_i $, the more important the page $ i $ is in the network.

---


### **Teleportation**

In the **web graph**, for example, we can find a web page **i** which refers only to web page **j** and **j** refers only to **i**. This is what we call **spider trap problem**. We can also find a web page which has no outlink. It is commonly named **Dead end**.

In the case of a **spider trap**, when the **random walker** reaches the **node 1*8 in the above example, he can only jump to **node 2** and from **node 2**, he can only reach **node 1**, and so on. The importance of all other nodes will be taken by **nodes 1** and **2**. In the above example, the probability distribution will converge to $\pi = (0, 0.5, 0.5, 0)$. This is not the desired result.

In the case of **Dead ends**, when the walker arrives at **node 2**, it can’t reach any other node because it has no outlink. The algorithm cannot converge.

To get over these two problems, we introduce the notion of **teleportation**.

**Teleportation** consists of connecting each node of the graph to all other nodes. The graph will be then complete. The idea is with a certain probability $\beta$, the **random walker** will jump to another node according to the **transition matrix P** and with a probability **(1-β)/n**, it will jump randomly to any node in the graph. We get then the new **transition matrix R**


$$ R = \beta P +(1-\beta)ve^T$$

where **v** is a vector of ones, and **e** a vector of **1/n**:

$$ e^T = (\frac{1}{n},\cdots,\frac{1}{n}) $$


$$ v = (1,\cdots,1)^T $$

$\beta$ is commonly defined as the **damping factor**


The  **matrix R** has the same properties than **P** which means that it admits a stationary distribution, so we can use all the theorems we saw previously.

# **Algorithms**

## **PageRank**

**PageRank** was created to tacke the difficulty of determing the importance of web pages with immense amount of information available.Demonstrates how pure math can have massive practical impact — affecting **billions of users** every day.The purpose of the algorithm is to provide better search results that are more precise and related by taking into account various factors beyond just matching keywords.How i mentonied before most search engines focused only on matching words, not on evaluating the importance of pages.PageRank introduced a completely new way of thinking about web pages. Instead of treating every page as equal, it looked at how other pages linked to it.
A page is important if important pages link to it.
How before we told that **PageRank** algorithm is based on the idea of a **random surfer** — someone who starts on a web page and randomly follows links.
Sometimes they click a link on the page or get bored and jump to a random new page.
By simulating this behavior across the web, PageRank calculates the steady-state probability of landing on any given page. That value becomes the page’s ranking score.

<center>
    <img src="https://th.bing.com/th/id/OIP.gazojYMdRdFHxk9FqT3OQAHaF9?r=0&rs=1&pid=ImgDetMain" alt="PageRank" width="300" height="200">
</center>

## **Basic Idea**
To illustrate how **PageRank** works i will show simple way of explanation.

Let's use players in a football match:

- each player represent a page

- each pass between two players represent a link between 2 pages



<center>
    <img src="https://thumbs.dreamstime.com/b/vector-soccer-field-arrangement-players-game-position-title-football-player-green-field-template-vector-119751372.jpg" alt="Image of football field with red points for player" width="300" height="200">
</center>

The main thing PageRank uses are:
  - The number of links the page gets(in football context how many pass player recieves).
    
  - The importance of a page is determined by the number of links pointing towards it by how frequently the player who passed the ball is passed to.
The **PageRank** of each player gets updated every time they receive the ball.

- As more passes are made ,the PageRank of each player undergoes changes.

- As a result ,the **PageRank** of every player they pass will be altered.

- **The higher number it's better**.

- Once game concludes,players can be **sorted** by their rating and be **ranked** to determine the best performer!


As this very simple explanation we hover the main idea of **PageRank Algorithm**!

---

## **Algorithm**

The PageRank algorithm outputs a probability distribution used to represent the likelihood that a person randomly clicking on links will arrive at any particular page. PageRank can be calculated for collections of documents of any size. It is assumed in several research papers that the distribution is evenly divided among all documents in the collection at the beginning of the computational process. The PageRank computations require several passes, called **iterations**,through the collection to adjust approximate PageRank values to more closely reflect the theoretical true value.

A probability is expressed as a numeric value between 0 and 1. A 0.5 probability is commonly expressed as a "50% chance" of something happening. Hence, a document with a PageRank of 0.5 means there is a 50% chance that a person clicking on a random link will be directed to said document.


---

Lets have small universe of four web pages: **A**, **B**, **C**, and **D**. Links from a page to itself are ignored. Multiple outbound links from one page to another page are treated as a single link. **PageRank** is initialized to the same value for all pages. In the original form of **PageRank**, the sum of **PageRank** over all pages is the total number of pages on the web at that time.However, later versions of **PageRank**, and the remainder of this section, assume a probability distribution between **0** and **1**. Hence the initial value for each page in this example is **0.25**.

The PageRank transferred from a given page to the targets of its outbound links upon the next iteration is divided equally among all outbound links.

If the only links in the system were from pages **B**, **C**, and **D** to **A**, each link would transfer **0.25** PageRank to **A** upon the next iteration, for a total of 0.25 + 0.25 + 0.25 = **0.75**.

$$ PR(A) = PR(B) + PR(C) + PR(D). $$

Instead in our example page **B** had a link to pages **C** and **A**, page **C** had a link to page **A**, and page **D** had links to all three pages. In the first iteration, page **B** would transfer half of its existing value **(0.125)** to page **A** and the other half **(0.125)** to page **C**. Page **C** would transfer all of its existing value **(0.25)** to the only page it links to, **A**. Since **D** had three outbound links, it would transfer one third of its existing value, or approximately **0.083**, to **A**. At the completion of this iteration, page **A** will have a PageRank of approximately **0.458**.

$$ PR(A) = \frac{PR(B)}{2} + \frac{PR(C)}{1} + \frac{PR(D)}{3}. $$

In other words, the PageRank conferred by an outbound link is equal to the document's own PageRank score divided by the number of outbound links **L( )**.

$$ PR(A) = \frac{PR(B)}{L(B)} + \frac{PR(C)}{L(C)} + \frac{PR(D)}{L(D)}. $$

In general case PageRank value for a page **U** is dependent on the PageRank values for each page **V** contained in the set **Bu** (the set containing all pages linking to page **U**), divided by the number **L(V)** of links from page **V**.

$$ PR(U) = \sum_{v \in B_u} \frac{PR(V)}{L(V)} $$

The damping factor is subtracted from 1 (and in some variations of the algorithm, the result is divided by the number of documents (N) in the collection) and this term is then added to the product of the damping factor and the sum of the incoming PageRank scores.

$$  PR(A) = \frac{1-d}{N} + d\left(\frac{PR(B)}{L(B)} + \frac{PR(C)}{L(C)} + \frac{PR(D)}{L(D)} + \cdots \right) $$

The damping factor adjusts the derived value downward. The original paper, however, gave the following formula, which has led to some confusion:

$$  PR(A) = 1-d + d\left(\frac{PR(B)}{L(B)} + \frac{PR(C)}{L(C)} + \frac{PR(D)}{L(D)} + \cdots \right) $$

The difference between them is that the PageRank values in the first formula sum to one, while in the second formula each PageRank is multiplied by N and the sum becomes N. A statement in Page and Brin's paper that the sum of all PageRanks is one and claims by other Google employees support the first variant of the formula above.

Page and Brin confused the two formulas in their most popular paper "The Anatomy of a Large-Scale Hypertextual Web Search Engine", where they mistakenly claimed that the second formula formed a probability distribution over web pages.

Google recalculates PageRank scores each time it crawls the Web and rebuilds its index. As Google increases the number of documents in its collection, the initial approximation of PageRank decreases for all documents.

The formula uses a model of a random surfer who reaches their target site after several clicks, then switches to a random page. The PageRank value of a page reflects the chance that the random surfer will land on that page by clicking on a link. It can be understood as a **Markov chain** in which the **states** are **pages**, and the **transitions** are the **links** between pages – all of which are all equally probable.

---
When calculating PageRank, pages with no outbound links are assumed to link out to all other pages in the collection. Their PageRank scores are therefore divided evenly among all other pages. In other words, to be fair with pages that are not sinks, these random transitions are added to all nodes in the Web. This residual probability, **d**, is usually set to **0.85**, estimated from the frequency that an average surfer uses his or her browser's bookmark feature. So, the equation is as follows:

$$  PR(pi\text) = \frac{1-d}{N} \sum_{pj \in M(pi\text)} \frac{PR(pi\text)}{L(pi\text)} $$

where $ p_1,p_2,...,p_n$ are pages under consideration,$ M(pi\text)$ is the set of pages that link $ pi\text ,L(pi\text)$ is the number of outbond links on page $ p_j$ , and **N** is the total number of pages.

The PageRank values are the entries of the dominant right **eigenvector** of the modified adjacency matrix rescaled so that each column adds up to one. 

So this is the main formula of  **PageRank** algorithm!

---

The **PageRank** values are the entries of the dominant right eigenvector of the **modified adjacency matrix** rescaled so that each column adds up to one. 
This makes PageRank a particularly elegant metric: the eigenvector is


$$ R = \begin{bmatrix}
PR(p_1) \\
PR(p_2)  \\
\vdots  \\
PR(p_N)
\end{bmatrix}$$

where **R** is the solution of the equtation

$$ R=\begin{bmatrix}
(d-1)/N \\
(d-1)/N \\
\vdots  \\
(d-1)/N
\end{bmatrix} + d\begin{bmatrix}\ell(p_1,p_1) & \ell(p_1,p_2) & \cdots &  \ell(p_1,p_N)\\ 
\ell(p_2,p_1) & \ddots &  \space & \vdots  \\
\vdots & \space & \ell(p_i,p_j) & \space \\
\ell(p_N,p_1) & \cdots & \space & \ell(p_N,p_N)  \end{bmatrix}R$$

where adjency function $ \ell(p_i,p_j)$ is the ratio between number of links outbound from page **j** to page **i** to the total number of outbound links of the page **j**.The adjency function is 0 if page $p_j$ does not link $ p_i$,and normalized that for each j:

$$ \sum_{i=0}^{N} \ell(p_i,p_j) = 1 $$

Elements of each columns is sum to **1**,so matrix is a **stochastic metrix**.It's a variant of **eigenvector** centrality measure used commonly in network analysis.Because of the large **eigengap** of the modified adjacency matrix above,the values of the PageRank eigenvector can be approximated to within a high degree of accuracy within only a few iterations.

Through this data, algorithm can be scaled very well and that the scaling factor for extremely large networks would be roughly linear in 
$\log n$ where **n** is the size of the network.

## Computation Summary

At $ t = 0 $ ,an initial probability distribution is asummed usually

$$ PR(pi;0) = \frac{1}{N}$$.

where **N** is the total number of pages and $pi;0$ is a page $ i$ at time 0.

At each time step,the computation,as detailed above yields

$$ PR(pi;t+1) = \frac{1-d}{N} + d  \sum_{pj \in M(pi\text)} \frac{PR(pj;t)}{L(pj)} $$

where **d** is dumping factor or in matrix notation **->**

$$ R(t + 1) = dMR(t)+\frac{1-d}{N}1$$.

where $ R_i(t) = PR(pi;t)$ and **1** is column vector of length **N** containing only ones.

The Matrix **M** is defined as:

$$ M_{ij} =
\begin{cases}
{1}/{L(p_j)}, & \text{if j links to i } \\
0, & \text{otherwise}
\end{cases} $$

Same as 

$$  M = (K^{-1}A)^T$$

where 
- **A** denotes the adjacency matrix of the graph
- **K** This is a diagonal matrix where each diagonal entry  is the outdegree of page (how many pages it links to).

$$
K = 
\begin{bmatrix}
2 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{bmatrix}
$$

But $ K^{-1}$ is inverse of $K$

$$
K^{-1} = 
\begin{bmatrix}
\frac{1}{2} & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{bmatrix}
$$

We want a transition matrix 𝑀 where 𝑀𝑖𝑗  = probability of moving from page j to page i
To get that, we:
- Normalize each column of 𝐴 by its outdegree
- Transpose to match the direction of movement

This creates **transition probability matrix**!

The probability calculation is made for each page at a time point, then repeated for the next time point. The computation ends when for some small **ϵ**!
$$ |R(t+1) - R(t)| < ϵ  $$
This is callled **convergence**!

---

##  **Implementation with Python**

First I will implement **PageRank algoritm** with simplier way using **networkx** library which has integrate inside **nx.pagerank()** function  with parameter graph and damping factor!After that i will do pure Python **PageRank** algorithm! 

In [ ]:
# load CSV file which is simple web graph of 500 pages with random links
df = pd.read_csv("simple_web_graph.csv")

# Create a directed graph
G = nx.DiGraph()

# Add edges from the CSV
edges = list(zip(df["source"],df["target"]))
G.add_edges_from(edges)

# Run PageRank

pagerank_scores = nx.pagerank(G,alpha=0.85)

# Convert to DataFrame and sort by rank

pagerank_df = pd.DataFrame(pagerank_scores.items(), columns=["Page", "Score"])
pagerank_df = pagerank_df.sort_values(by="Score", ascending=False)




In [ ]:
# Visualize a pages  of the graph with this function!

def visulize_graph(G,pagerank_df,pages,title):
    small_graph = G.subgraph(list(pagerank_df.head(pages)["Page"]))
    
    plt.figure(figsize=(10, 6))
    nx.draw(small_graph, with_labels=True, node_size=700, node_color='lightblue', arrows=True)
    plt.title(title)
    plt.show()

In [ ]:
# Visuallise top 10 pages!
visulize_graph(G,pagerank_df,10,"Top 10 pages by PageRank algoritm!")

In [ ]:
# Visuallise top 30 pages!
visulize_graph(G,pagerank_df,30,"Top 30 pages by PageRank algoritm!")

In [ ]:
# Visuallise 50 pages!
visulize_graph(G,pagerank_df,50,"Top 50 pages by PageRank algoritm!")

## PageRank algorithm function

In [ ]:
def pagerank(df,alpha=0.85, tol=1e-6, max_iter=100):
    pages = sorted(set(df['source']).union(set(df['target'])))
    page_to_index = {page: i for i, page in enumerate(pages)}
    index_to_page = {i: page for page, i in page_to_index.items()}
    n = len(pages)

    #Initilize Transition Matrix
    M = np.zeros((n,n))

    #Fill the matrix based on links
    for row in df.itertuples():
        src = page_to_index[row.source]
        tgt = page_to_index[row.target]
        M[tgt,src] += 1 # Column-stochastic matrix

    #Hanle Dangling Nodes Too
    for i in range(n):
     col_sum = M[:, i].sum()
     if col_sum != 0:
        M[:, i] /= col_sum
     else:
        # Handle dangling node (pages with no outbound links): use uniform distribution 
        M[:, i] = 1.0 / n

     

    matrix_shape = M.shape[0]
    v = np.ones(matrix_shape) / matrix_shape # Intitial Rank
    teleport = np.ones(matrix_shape) / n  #Teleportation Vector


    for _ in range(max_iter):
        v_new = alpha * np.matmul(M, v) + (1 - alpha) * teleport
        if np.linalg.norm(v_new - v, 1) < tol:
            break
        v = v_new

    # Result is a stationary distribution vector — each entry is the PageRank of that page.

    return v
    

### **Compare Results** 

In [ ]:
%timeit pagerank(df)

In [ ]:
%timeit nx.pagerank(G,alpha=0.85)

### **Damping Factor Change**

**Low damping** → More randomness (less importance to link structure)

**High damping** → More importance to actual links

In [ ]:
damping_values = [0.3,0.4,0.5, 0.7, 0.85, 0.95]

# Store all PageRank results for each damping factor
all_ranks = []

# Run pagerank for each damping factor and collect the rank arrays
for d in damping_values:
    ranks = pagerank(df, alpha=d)  # returns NumPy array
    all_ranks.append(ranks)

# Convert list of arrays into a 2D NumPy array for easier indexing [damping_index][page_index]
rank_matrix = np.array(all_ranks)

# Plot PageRank scores across damping values
plt.figure(figsize=(12, 6))

num_pages = rank_matrix.shape[1]  # number of pages

# Plot for each page
for i in range(min(10,num_pages)):
    plt.plot(damping_values, rank_matrix[:, i], marker='o', label=f'Page {i}')

plt.title("PageRank Score vs Damping Factor")
plt.xlabel("Damping Factor")
plt.ylabel("PageRank Score")
plt.grid(True)

# If too many pages, you can comment this line or limit the number of pages plotted
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize="small")

plt.tight_layout()
plt.show()

## **Personalized PageRank**
Personalized PageRank (PPR) is an adaptation of the classic PageRank algorithm that personalizes the ranking results for a specific user, query, or topic. Instead of treating all nodes equally during teleportation, PPR biases the random surfer toward a personalized subset of nodes.

In contrast to classic PageRank (where teleportation happens uniformly to any node), PPR allows the teleportation step to prefer specific nodes based on interest, relevance, or preference.

$$ 
\vec{r} = \alpha M \vec{r} + (1 - \alpha) \vec{v}
$$



## **Implementation of Personlized PageRank**

In [ ]:
def personalized_pagerank(G, alpha=0.85, personalization_node=None, max_iter=100, tol=1e-6):
    nodes = list(G.nodes())
    N = len(nodes)
    A = nx.to_numpy_array(G, nodelist=nodes)
    
    # Create transition matrix M
    M = A / A.sum(axis=1, keepdims=True)
    M = np.nan_to_num(M)  # handle dangling nodes
    
    # Personalization vector v
    v = np.zeros(N)
    if personalization_node is not None:
        index = nodes.index(personalization_node)
        v[index] = 1.0
    else:
        v[:] = 1.0 / N
    v = v / v.sum()  # ensure it’s a distribution
    
    # Initialize pagerank vector
    p = np.ones(N) / N
    
    for i in range(max_iter):
        p_new = alpha * M.T @ p + (1 - alpha) * v
        if np.linalg.norm(p_new - p, 1) < tol:
            break
        p = p_new
    
    return dict(zip(nodes, p))


In [ ]:
G = nx.DiGraph()
edges = [("A", "B"), ("A", "C"), ("B", "C"), ("C", "A"), ("D", "C")]
G.add_edges_from(edges)

result = personalized_pagerank(G, alpha=0.85, personalization_node="A")
print("Personalized PageRank starting from node A:")
for node, score in result.items():
    print(f"{node}: {score:.4f}")

## **HITS Algorithm(_Hyperlink Induced Topic Search_)**

**Hyperlink Induced Topic Search (HITS) Algorithm** is a Link Analysis Algorithm that rates webpages, developed by Jon Kleinberg. This algorithm is used to the web link-structures to discover and rank the webpages relevant for a particular search.


Unlike **PageRank**, which assigns a single score to each page representing its importance, **HITS** assigns two scores to every page:

- **Authority Score**: Measures the value of the content on the page. A good authority page is one that is linked to by many good hubs.

- **Hub Score**: Measures the value of the links on the page. A good hub page links to many good authorities.


It use hubs and authorities to define a recursive relationship between webpages. 
 
- Given a query to a Search Engine, the set of highly relevant web pages are called Roots. They are potential **Authorities**.
- Pages that are not very relevant but point to pages in the Root are called **Hubs**. Authority is a page that many hubs link to whereas a Hub is a page that links to many authorities.

### **How it works?**

**HITS** is based on the idea that good hubs point to good authorities, and good authorities are pointed to by good hubs. The algorithm operates on a subset of the web graph related to a specific query:
A small subgraph (called a root set) is first created by collecting the top pages that match the query using a basic search.
This set is expanded by including pages that link to or are linked from the root set (forming a base set).
The hub and authority scores are then computed iteratively:

- Each node's authority score is the sum of the hub scores of the nodes pointing to it.

- Each node's hub score is the sum of the authority scores of the nodes it points to.

These scores are typically normalized in each iteration, and the process repeats until the scores converge.


## **Algorithm**

Let's have a directed graph $ G = (V,E)$ , where:

- $V$ is a set of nodes
- $E$ is set of directed edges(hyperlinks)

And have **adjacency matrix** representing links between pages.$A \in \mathbb{R}^{n×n}$

$ A_{ij}=1$ if page i link to page j,else 0.

$ h = \mathbb{R}^{n}$ hub score vector

$ a = \mathbb{R}^{n}$ authority score vector

Update rules are:

$$ a = A^{T}h $$ (authority is sum of hub scores pointing to it)

$$ h = Aa$$ (hub is sum of authority scores it points to)

Combining:

$$ a^{(k + 1)} = A^{T}Aa^{(k)}$$ 


$$ h^{(k + 1)} = AA^{(T)}h^{(k)}$$ 

So: 

$ A^TA $ is the authority matrix!

$ AA^T $ is the hub matrix!

These are symmetric positive semi-definite matrices, and the iterative update converges to the principal eigenvector of each matrix:

- The principal eigenvector of $A^TA$ gives authority scores.

- The principal eigenvector of $AA^T$ gives hub scores.

### **What this mean practically?**

- You can compute **authority scores** by finding the top eigenvector of $A^{T}A $ .
- You can compute **hub scores** by finding the top eigenvector of $ AA^{T}$.

    
  

-  Let number of iterations be **k**. 
-  Each node is assigned a **Hub score = 1** and an **Authority score = 1.**
- Repeat **k** times: 
 


- **Hub update** : Each node's Hub score = Σ  (Authority score of each node it points to).

- **Authority update** : Each node's Authority score = Σ  (Hub score of each node pointing to it).


Normalize the scores by dividing each Hub score by square root of the sum of the squares of all Hub scores, and dividing each Authority score by square root of the sum of the squares of all Authority scores. (optional)



## **Implementation with Python**

In [ ]:
def hits_algorithm(adj_matrix, max_iter=100, tol=1e-6):
    """
    Computes hub and authority scores using the HITS algorithm.
    
    Parameters:
        adj_matrix (numpy.ndarray): Adjacency matrix of the graph (shape: n x n)
        max_iter (int): Maximum number of iterations
        tol (float): Convergence tolerance
    
    Returns:
        hubs (numpy.ndarray): Hub scores
        authorities (numpy.ndarray): Authority scores
    """
    n = adj_matrix.shape[0]

    # Initialize hub and authority scores to 1
    hubs = np.ones(n)
    authorities = np.ones(n)

    for i in range(max_iter):
        # Update authority scores
        new_authorities = adj_matrix.T @ hubs
        # Update hub scores
        new_hubs = adj_matrix @ new_authorities

        # Normalize
        new_authorities = new_authorities / np.linalg.norm(new_authorities, 2)
        new_hubs = new_hubs / np.linalg.norm(new_hubs, 2)

        # Check convergence
        if np.allclose(hubs, new_hubs, atol=tol) and np.allclose(authorities, new_authorities, atol=tol):
            print(f"Converged in {i+1} iterations.")
            break

        hubs = new_hubs
        authorities = new_authorities

    return hubs, authorities

In [ ]:
# Example graph (5 pages)
A = np.array([
    [0, 1, 1, 0, 0],  # Page 0 links to 1, 2
    [0, 0, 1, 1, 0],  # Page 1 links to 2, 3
    [1, 0, 0, 0, 1],  # Page 2 links to 0, 4
    [0, 0, 0, 0, 1],  # Page 3 links to 4
    [0, 0, 1, 0, 0]   # Page 4 links to 2
])

hubs, authorities = hits_algorithm(A)

print("Hub Scores:", hubs)
print("Authority Scores:", authorities)

In [ ]:
pages = np.arange(len(hubs))

plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.bar(pages, hubs, color='skyblue')
plt.title("Hub Scores")
plt.xlabel("Page")
plt.ylabel("Score")

plt.subplot(1, 2, 2)
plt.bar(pages, authorities, color='salmon')
plt.title("Authority Scores")
plt.xlabel("Page")

plt.tight_layout()
plt.show()

## **Implementation with Networkx Module** 

Let's have **3** iterations $k=3$ (without Normalization)
I will display first changes for every iteration ->
## 🔢 Initial Hub and Authority Scores

| Node | Hub Score | Authority Score |
|------|-----------|-----------------|
| A    | 1         | 1               |
| B    | 1         | 1               |
| C    | 1         | 1               |
| D    | 1         | 1               |
| E    | 1         | 1               |
| F    | 1         | 1               |
| G    | 1         | 1               |
| H    | 1         | 1               |

---

## 🔁 After 1st Iteration

| Node | Hub Score | Authority Score |
|------|-----------|-----------------|
| A    | 1         | 3               |
| B    | 2         | 2               |
| C    | 1         | 4               |
| D    | 2         | 2               |
| E    | 4         | 1               |
| F    | 1         | 1               |
| G    | 2         | 0               |
| H    | 1         | 1               |

---

## 🔁 After 2nd Iteration

| Node | Hub Score | Authority Score |
|------|-----------|-----------------|
| A    | 2         | 4               |
| B    | 5         | 6               |
| C    | 3         | 7               |
| D    | 6         | 5               |
| E    | 9         | 2               |
| F    | 1         | 4               |
| G    | 7         | 0               |
| H    | 3         | 1               |

---

## 🔁 After 3rd Iteration

| Node | Hub Score | Authority Score |
|------|-----------|-----------------|
| A    | 5         | 13              |
| B    | 9         | 15              |
| C    | 4         | 27              |
| D    | 13        | 11              |
| E    | 22        | 5               |
| F    | 1         | 9               |
| G    | 11        | 0               |
| H    | 4         | 3               |


In [ ]:
G = nx.DiGraph()

G.add_edges_from([('A', 'D'), ('B', 'C'), ('B', 'E'), ('C', 'A'),
                  ('D', 'C'), ('E', 'D'), ('E', 'B'), ('E', 'F'),
                  ('E', 'C'), ('F', 'C'), ('F', 'H'), ('G', 'A'), 
                  ('G', 'C'), ('H', 'A')])

plt.figure(figsize =(10, 10))
nx.draw_networkx(G, with_labels = True)

hubs, authorities = nx.hits(G, max_iter = 50, normalized = True)
# The in-built hits function returns two dictionaries keyed by nodes
# containing hub scores and authority scores respectively.

print("Hub Scores: ", hubs)
print("Authority Scores: ", authorities)

## **BM25 (_Best Matching 25_**)

## **Algorithm**

BM25 is a ranking function that scores how relevant a document is to a search query, based on the occurrence of query terms in the document and it improves upon TF-IDF by introducing saturation and document length normalization.
When it comes to search engines and information retrieval, a vital piece of the puzzle is ranking the relevance of documents to a given query. One of the most widely used algorithms to achieve this is the BM25, Best Matching 25. BM25 is a probabilistic retrieval function that evaluates the relevance of a document to a search query, balancing simplicity and effectiveness, making it a popular choice in modern search engines and applications.
BM25 is essentially a scoring function that calculates a numerical score to estimate the relevance of a document for a given query. This score is based on the occurrences and importance of query terms within the document. The higher the score, the more relevant the document is considered to be.

$$  \sum^{n}_{i= 1}\text{IDF}(qi).\frac{f(q_i,D).(k_1 + 1)}{f(q_i,D) + k_1.(1-b + b.\frac{avgDL}{|D|})} $$

$( q_i )$ is a term in the query.


$f(q_i, D) )$ is the frequency of term $(q_i)$ in document $D$.


$|D|$ is the length of document $D$ (number of words).


$avgDL$ is the average document length in the corpus.


$k_1$ and $b$ are free parameters, where:

$k_1$ controls the term frequency saturation, with typical values around 1.2 to 2.

$b$ controls the degree of document length normalization, typically set to 0.75.
$\text{IDF}(q_i)$ is the inverse document frequency of term $q_i$.

**IDF function**:

$$ IDF = \log \frac{\text{Total number of documents in D}}{\text{Number of documents containing term}}$$

The **IDF function** adjusts term weight based on its distribution across documents, giving more importance to rarer terms.

### **Key Concepts**
BM25 improves upon traditional retrieval models by considering several essential aspects:

- Term Frequency (TF): The frequency of a term in the document directly impacts relevance. BM25, however, includes a saturation factor, meaning that additional occurrences of a term add less weight past a certain point, avoiding overemphasis on highly frequent terms.
- Inverse Document Frequency (IDF): BM25 uses IDF to balance term frequency by considering how rare or common a term is across documents in the corpus. This way, unique terms in a document are weighted more heavily than common terms.
- Document Length Normalization: BM25 incorporates a normalization factor to control the impact of document length on term frequency, which ensures that long documents are not unfairly penalized or favored.
- Adjustable Parameters (k1​ and b): BM25 allows flexibility with its two main parameters:
k1​ adjusts term frequency scaling, with higher values meaning more emphasis on term frequency.
b is a document length normalization parameter that allows BM25 to adapt to different types of datasets.




## **Implementation with Python**

In [ ]:
k1 = 1.5
b = 0.75

# Example document collection and query
corpus = [
    "the brown fox jumped over the brown dog",
    "the lazy dog sat in the sun",
    "the quick brown fox leaped over the lazy dog"
]
query = ["brown", "fox"]

# Pre-compute average document length
avg_doc_length = sum(len(doc.split()) for doc in corpus) / len(corpus)

# Function to calculate term frequency in a document
def term_frequency(term, document):
    return document.split().count(term)

# Function to calculate document frequency for a term
def document_frequency(term, corpus):
    return sum(1 for doc in corpus if term in doc)

# BM25 function
def bm25_score(query, document, corpus):
    score = 0
    doc_length = len(document.split())
    for term in query:
        tf = term_frequency(term, document)
        df = document_frequency(term, corpus)
        idf = math.log((len(corpus) - df + 0.5) / (df + 0.5) + 1)
        score += idf * ((tf * (k1 + 1)) / (tf + k1 * (1 - b + b * (doc_length / avg_doc_length))))
    return score

# Calculate BM25 scores for each document
for doc in corpus:
    print(f"Document: {doc}")
    print(f"BM25 Score: {bm25_score(query, doc, corpus)}")

---

This project was made by **Radoslav Todorov** as final project of the course **Math Concepts for Developers** organized by **SoftUni**!

## 📚 References
- Brin, Sergey, and Lawrence Page. *The Anatomy of a Large-Scale Hypertextual Web Search Engine*, Computer Networks and ISDN Systems, 1998. [link](http://ilpubs.stanford.edu:8090/361/1/1998-8.pdf)
- Geeksforgeeks *PageRank*  [link](http://ilpubs.stanford.edu:8090/361/1/1998-8.pdf)
- Geeksforgeeks *HITS* [link](https://www.geeksforgeeks.org/hyperlink-induced-topic-search-hits-algorithm-using-networkx-module-python/)
- *PageRank*. Wikipedia, [link](https://en.wikipedia.org/wiki/PageRank)
- *PageRank*. , [link](https://towardsdatascience.com/pagerank-algorithm-fully-explained-dc794184b4af/)
- *HITS*.Wikipedia,[link](https://en.wikipedia.org/wiki/HITS_algorithm)
- BM25 [link](https://learncodecamp.net/bm-25/)
- Search Engines,Wikipedia [link](https://en.wikipedia.org/wiki/Search_engine)
- Search Engines-[link](https://developers.google.com/search/docs/fundamentals/how-search-works)